In [1]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s

In [2]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
import os
import pinecone
from google.colab import userdata

# API key from app.pinecone.io and environment from console
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = userdata.get('PINECONE_ENVIRONMENT')

pinecone.init(
    api_key=os.environ.get(PINECONE_API_KEY) or PINECONE_API_KEY,
    environment=os.environ.get(PINECONE_ENVIRONMENT) or PINECONE_ENVIRONMENT
)

In [5]:
docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 384.


In [8]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

In [9]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

In [10]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [11]:
vectorstore.add_texts(["More text to add as an example!"], [{'name':'example'}])

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

['a1c7eb50-5a3a-4c9a-9103-42aae2dcd17b']

In [ ]:
vectorstore.similarity_search(
    "text to add as an example",  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!git clone https://github.com/MahaJayapal/LLM.git
dataset = '/content/LLM/dataset'

Cloning into 'LLM'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 8), reused 15 (delta 3), pack-reused 0
Receiving objects: 100% (28/28), 70.53 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [18]:
def ingest_data(data_dir):
  for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        # Read text file
        with open(os.path.join(data_dir, filename), "r") as f:
            text = f.read()
        metadata = {'filename': filename, 'Release-notes': 'week-of-september-29-2023'}
        vectorstore.add_texts([text], [metadata])


In [15]:
ingest_data(dataset)

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
vectorstore.similarity_search(
    "additional ChangeDetection and Forecast",  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Release-notes: week-of-september-29-2023\nAPI Framework Enhancements\nAPI updates\nAPIs to search and assign templates have been updated to support additional SaaS Application collector type. See API documentation for more information.\nAPIs to manage templates have been updated to support the additional ChangeDetection and Forecast threshold types. See API documentation for more information.', metadata={'name': 'example'}),
 Document(page_content='uponthecontext. AsindicatedinFigure8,thetemperatureappearstobeinﬂuencedbyRLHF.Yet,intriguingly,\nour ﬁndings also revealed that the shifts are not uniformly applied across all prompts, as shown in Figure 21.\nForinstance,whenitcomestopromptsassociatedwithcreativity,suchas“Writeapoem,”anincreasein\ntemperature continues to generate diversity across our various RLHF iterations. This can be observed in the\nSelf-BLEU slope, which mirrors a pattern comparable to that of the SFT model.\nOntheotherhand,forpromptsbasedonfact